This python code look at a 4 year provincial NDWI time serie and produces several raster outputs including:
    -NDWI anomaly
    -NDWI standard deviation
    -NDWI mean
    -NDWI RGB of last 3 days

### 1. Import python libraries

In [2]:
import pandas as pd
import pandas as pd
import gdal
import ogr, osr
from osgeo import gdal
from gdalconst import *
import numpy as np
import time,os, glob,sys,datetime
import subprocess, shutil
import matplotlib.pylab as plt
#import subprocess as sp
import tempfile
import matplotlib.patches as patches #needed for drawing rectangles
#from skimage import exposure #enhance image display
gdal.UseExceptions()

### 2. Create processor class with some methods

In [3]:
#some PREPARARATION WORK
#****************************************************
# 1. CREATE DIRECTORIES
#*******************************************************88
# # Set DryRun to True for a list of the gdalbuildvrt commands only without making the call
DryRun=False

class create_NDWI(object):
    ''' for Modis zone  h11v03, this module reads an .vrt mosaic file from the main_folder = fold + '/main/
        folder. for every run it produces:
         - a global mean grid 
         - a stand deviation grid
         - an anomaly grid for current image
         - a RGB image from last three NDWI images

                            ...........of NDWI for period January 2017 till today.

        '''

    # # 1.a)  CREATE FUNCTION TO MAKE A TIMESERIES
    def save_vrt2tif(self, output_vrt,tifname):
        gdal_translate = r'C:/PolderNaemL/gdal_translate.exe'
        # save virt as tif

        #cmd = '-of GTiff -projwin 198000 6668000 847000 6262000'
        cmd = '-of GTiff'
        tran_cmd = ' '.join([gdal_translate , cmd, output_vrt, tifname])
        print("transcom:", tran_cmd)
        subprocess.Popen(tran_cmd)
       # return tifname

    def append_date(self, adatum, date_file):
        ''' when we do updates with a new file
        it opens a file with dates *.dates and append
        a new date to .dates file'''
        with open(date_file, 'a') as f:
            #f.write(f'\n{adatum}')
            f.write('\n(adatum')

    def clean_folder(self,folder, format='.tif'):
        ''' removes all tiff files from folder'''
        files = glob.glob(folder +'/*' + format)
        for f in files:
            os.remove(f)
        return
    def mkdir(self,dirname, remove=True, chdir=True):
        import shutil
        """create a directory dirnme.  if it iexists     , it is removed by shutil.rmtree
        """
        if os.path.isdir(dirname):
            if remove:
                shutil.rmtree(dirname)
            else:
                return False  # did not make new directory
        os.mkdir(dirname)

        return
    def vrt_mosac(self,output_vrt, dir ):
        ''' create vrt band to update main file'''
        buildvrt = r'C:/olderNaemL/gdalbuildvrt.exe'

        cmd ="-separate " + output_vrt +' '+ dir + '/*.tif'
        #cmd = output_vrt + ' ' + dir
        fullCmd = ' '.join([buildvrt, cmd])
        print("fullcmd:", fullCmd)
        subprocess.Popen(fullCmd)
        print("output file is done")
        return
    def create_RGB(self,input_vrt,out_file, bands):
        #gdal_translate - of GTiff - b 154 -b 155 - b 156 main_NDWI_122_.vrt main_NDWI_122_RGB.tif
        gdal_translate = r'C:/olderNaemL/gdal_translate.exe'
        # save virt as tif

        # cmd = '-of GTiff  ' +
        cmd = '-of GTiff -b ' + str(bands-2) + ' -b '+ str(bands-1)+ ' -b '+ str(bands)
        rgb_cmd = ' '.join([gdal_translate, cmd, input_vrt, out_file])
        print("rgbcom:", rgb_cmd)
        subprocess.Popen(rgb_cmd)
        # return tifname

        return

    def run_NDWI(self):
        # import os

        fold = 'C:/_LolderNaemTS_hist/prj_2020/AB_NDWI'
        os.chdir(fold)
        # Give the root directory where the subdirectories containing the  files are stored
               #
        # # 1.b)  cOPY CURRENT FILE TO ARCHIVED FOLDER
        # # Update the main file with adding curent file to the main
        cur_folder = fold + '/current_image'
        archive = fold +'/ARCHIVE'
        main_folder = fold + '/main'
        RGB_folder =fold + '/RGB'
        # List *.vrt files
        archive_list = glob.glob(fold + '/main/*.vrt')
        print("archived list: ", archive_list[0])

        try:

            #read a virtual mosaic.....
            ds = gdal.Open(archive_list[0], 0)
            if ds is None:
                print("no data set")
            # #get projection info
            ds_proj = ds.GetProjection()
            geot = ds.GetGeoTransform()
            width = ds.RasterXSize
            height = ds.RasterYSize
            bands = ds.RasterCount
            print(width, bands, height, bands-1)

            #ds1 = gdal.Open(imagefile, 0).ReadAsArray()
            ds1 = gdal.Open(archive_list[0], 0).ReadAsArray()
            if ds1 is None:
                print("no data set")
            bands,rows,cols = ds1.shape
            shp = [cols,rows]
            print("\nds1:", ds1.dtype, shp)
            today = datetime.datetime.now()
            day_of_year = (today - datetime.datetime(today.year, 1, 1)).days + 1
            # day of year goes into name of the output         
           
            # create s few numpy array ................
            value = np.zeros((rows, cols), dtype=np.float)
            sum_3year = np.zeros((rows, cols), dtype=np.float)
            
            # stand deviat for last 4 years
            std_devPP = np.zeros((rows, cols), dtype=np.float)
            # anomaly
            ndwi_anomal = np.zeros((rows, cols), dtype=np.float)
            
            # # # #**********************************************88
            # 3.a) calculate global mean  for 3year period **********************             
            # #*******************************************************
           
            # bands +1 are all bands. we exclude last one with "bands"
            for b in range(bands - 1):
           
                band = ds.GetRasterBand(b + 1)
                bands_data = np.array(band.ReadAsArray(), dtype=float)
                # cumulative sume of NDWI 8 day composites for 3 years EXCEP THE NEW TODAY'S LAST BAND
                value = (value + bands_data)
      
            np.seterr(divide='ignore', invalid='ignore')
            global_3ymean = value / (bands - 1)
            #
           # # # #**********************************************88
           # 3.b)  calculate Stdev for each pixel  *****************************88
            # #***************************************************************
            band1 = ds.GetRasterBand(1)
            band1_data = np.array(band1.ReadAsArray(), dtype=float)
            for i in range(rows):
                for j in range(cols):
                    # pix =band1_data[i][j]
                    stand = ds1[0:bands - 1, i, j]
                    # calculate Standard deviation np.std(arr, dtype = np.float32))
                    std_devPP[i][j] = np.std(stand)
                    # print("stdshp:",stand)
            ##DEFINE output names

            print("standDev: ", std_devPP[0][0])
            mean_name = 'output' + '/' + 'gloab_mean_' + str(day_of_year) + '.tif'
            std_name = 'output' + '/' + 'glob_std_' + str(day_of_year) + '.tif'
            anom_name = 'output' + '/' + 'anom_' + str(day_of_year) + '.tif'
            # #     #create output chanel in a tiff file
            outdriver = gdal.GetDriverByName("GTiff")
            out_mean = outdriver.Create(mean_name, shp[0], shp[1], 1, gdal.GDT_Float32)
            out_std = outdriver.Create(std_name, shp[0], shp[1], 1, gdal.GDT_Float32)

            # #get ouptup band and write global_mean data into
            nodatav = -9999
            out_mean.GetRasterBand(1).WriteArray(global_3ymean)
            #out_mean.GetRasterBand(1).SetNoDataValue(nodatav)
            # #
            # #get ouptup band and write std_mean data into
            out_std.GetRasterBand(1).WriteArray(std_devPP)
            #set no data value
            out_std.GetRasterBand(1).SetNoDataValue(nodatav)

            # #set space
            out_mean.SetGeoTransform(geot)
            out_std.SetGeoTransform(geot)

            # #set projection
            out_mean.SetProjection(ds_proj)
            out_std.SetProjection(ds_proj)

            # # # #**********************************************88
            # 3.c) calculate ANOMALY of ndwi and create output file
            # # #*****************************************88

            period = (bands + 1 - bands)           
            anom_driver = gdal.GetDriverByName("GTiff")
            out_anom = anom_driver.Create(anom_name, shp[0], shp[1], period, gdal.GDT_Float32)
            out_anom.SetGeoTransform(geot)
           # nodatav = -999
            for month in range(bands, bands + 1):
                data_after = np.array(ds.GetRasterBand(month).ReadAsArray(), dtype=float)
                ndwi_anomal = (data_after - global_3ymean) / std_devPP               
                # create output chanel in a tiff file
                sh = data_after.shape                
                out_anom.GetRasterBand(period).WriteArray(ndwi_anomal)
                #out_anom.GetRasterBand(period).SetNoDataValue(nodatav)
                # ensure that the data have been written to disk instead of only cached in memory,
                out_anom.FlushCache()
                # makes it easier for some software to display it nicely
                out_anom.GetRasterBand(period).ComputeStatistics(False)
                # build overview layers for the dataset
                out_anom.BuildOverviews('average', [2, 4, 8, 16, 32])
                ndwi_anomal = None
                data_after = None
            
            #set projection for anomaly file...
            out_anom.SetProjection(ds_proj)

            #create RGB file
            out_RGB_a = os.path.basename(archive_list[0]).replace('main','RGB')[:-4] + '_h11v03.tif'            
            out_RGB =os.path.join( RGB_folder ,out_RGB_a)           
            self.create_RGB( archive_list[0], out_RGB, bands)

            # close ouptut variable
            out_mean = None
            out_std = None
            anom_driver = None
            outdriver = None
            out_anom = None
            ds = None
            ds1 = None


        except IOError:
            print('Folder is not accessible')



In [ ]:
#run the file

if __name__ == '__main__':
    # create object
    obj_ndwi = create_NDWI()

    obj_ndwi.run_NDWI()
    obj = None
